In [5]:
import math
import nltk
import hashlib
import random
import spacy
import numpy as np
from random import seed
from nltk.wsd import lesk
from random import randint
from nltk.corpus import wordnet as wn
from nltk.corpus import framenet as fn
from nltk.corpus.reader.framenet import PrettyList

import common.prettyprint as pp 
import common.utils as utils

nlp = spacy.load('en_core_web_trf')

### 0. Individuazione di un set di Frame (FrameSet)

Il blocco seguente è stato preso (e in parte ri-adattato) dal notebook fornito a lezione per recuperare i frame dato un cognome.

In [7]:
def print_frames_with_IDs():
    for x in fn.frames():
        print('{}\t{}'.format(x.ID, x.name))

def get_frams_IDs():
    return [f.ID for f in fn.frames()]

def getFrameSetForStudent(surname, list_len=5):
    result = list()
    nof_frames = len(fn.frames())
    base_idx = (abs(int(hashlib.sha512(surname.encode('utf-8')).hexdigest(), 16)) % nof_frames)
    print('\nstudent: ' + surname)
    framenet_IDs = get_frams_IDs()
    i = 0
    offset = 0 
    seed(1)
    while i < list_len:
        fID = framenet_IDs[(base_idx+offset)%nof_frames]
        result.append(fID)
        f = fn.frame(fID)
        fNAME = f.name
        print('\tID: {a:4d}\tframe: {framename}'.format(a=fID, framename=fNAME))
        offset = randint(0, nof_frames)
        i += 1
    return result
    

In [8]:
frameset = getFrameSetForStudent("D'Amato")


student: D'Amato
	ID: 2630	frame: Standing_by
	ID:  880	frame: Being_in_operation
	ID:   15	frame: Separating
	ID:   44	frame: Volubility
	ID: 2659	frame: Distant_operated_IED


### 1. Consegna

Per ogni frame nel FrameSet è necessario assegnare un WN synset ai seguenti elementi:

- **Frame name** (nel caso si tratti di una multiword expression, come per esempio 'Religious_belief', disambiguare il termine principale, che in generale è il **sostantivo** se l'espressione è composta da NOUN+ADJ, e il **verbo** se l'espressione è composta da VERB+NOUN; in generale l'elemento fondamentale è individuato come il **reggente dell'espressione**.
- **Frame Elements (FEs)** del frame; e 
- **Lexical Units (LUs)**.

I contesti di disambiguazione possono essere creati utilizzando le definizioni disponibili (sia quella del frame, sia quelle dei FEs), ottenendo `Ctx(w)`, il contesto per FN terms `w`.

Per quanto riguarda il contesto dei sensi presenti in WN è possibile selezionare glosse ed esempi dei sensi, e dei loro rispettivi iponimi e iperonimi, in modo da avere più informazione, ottenendo quindi il contesto di disambiguazione `Ctx(s)`.

La seguente funzione sfrutta spacy (una libreria) per fare la parsificazione. Il reggente, ovvero la radice dell'albero a dipendenze (trovato durante il parsing), viene restituito.

In [9]:
def find_root(multiword_exp):
    doc = nlp(multiword_exp)
    for token in doc:
        if token.dep_ == 'ROOT':
            return token.text
    else:
        return None

Le seguenti funzioni sono state create per generare il contesto di un frame element.  
* `get_disambiguation_ctx_for_fn` crea il contesto basandosi unicamente sulla definizione del frame.
* `get_disambiguation_ctx_for_fes` crea il contesto basandosi sulle definizioni dei frame elements forniti in input
* `get_combined_ctx` crea il contesto come unione del risultato delle precedenti funzioni

In [10]:
def get_disambiguation_ctx_for_fn(frame):
    # Takes the frame definition, returns a Bag-of-words model
    return utils.preprocess_phrase_nostem(frame.definition)

def get_disambiguation_ctx_for_fes(frame_elements):
    # Takes the frame elements, returns a Bag-of-words model for the whole definitions
    result = list()
    for key in frame_elements.keys():
        result.append(utils.preprocess_phrase_nostem(frame_elements[key].definition))
    return utils.flatten(result)

def get_combined_ctx(frame):
    # Takes a frame, returns a Bag-of-Words model for all the definitions in the frame
    fn = get_disambiguation_ctx_for_fn(frame)
    fes = get_disambiguation_ctx_for_fes(frame.FE)
    return fn + fes

La seguente funzione genera il contesto di disambiguazione per un certo synset. Per farlo, si basa:  
1. Sulla definizione del synset fornito in input  
2. Sugli esempi del synset fornito in input  
3. Sulla definizione e esempi degli iperonimi diretti del synset fornito in input  
4. Sulla definizione e esempi degli iponimi diretti del synset fornito in input  

In [11]:
def get_big_disambiguation_ctx_for_sysnet(synset):
    # Given a synset, returns a Bag-of-Words model for its definition and examples
    # as well as those of the direct hypernyms and direct hyponyms
    context = synset.definition()
    for example in synset.examples():
        context += example

    for hypernym in synset.hypernyms():
        context += hypernym.definition()
        for hyp_example in hypernym.examples():
            context += hyp_example
    
    for hyponym in synset.hyponyms():
        context += hyponym.definition()
        for hypo_example in hyponym.examples():
            context += hypo_example
    return list(zip(*utils.word_frequencies(utils.preprocess_phrase_nostem(context))))[0]

La seguente funzione trova **tutti** i synsets associabili ad un certo termine (tipicamente il nome di un frame). Si può scegliere di effettuare una ricerca con spacy oppure una cieca (attraverso il parametro `method`):
* `method=spacy (default)`: Usa il parser di spacy per trovare il reggente della frase, dopodiché lo ricerca su WordNet fornendone i risultati in output
* `method=dumb`: Dato il termine in input, lo separa in sottotermini (`.split('_')`) e cerca ognuno di essi in WordNet, restituendone i risultati
Se per qualche ragione, il parametro 'method' non é specificato, non restituisce alcun risultato (`None`)

In [12]:
def get_synset_for_fn(frame_name: str, method='spacy'):
    # given a frame name, returns all the relevant synsets
    if method == 'spacy':
        root = find_root(frame_name.replace('_', ' '))
        return wn.synsets(root)
    elif method == 'dumb':
        strings = frame_name.split('_')
        synsets = [wn.synsets(string) for string in strings]
        return utils.flatten(synsets)
    return None

La prossima funzione é una specializzazione (più scarna) della precedente, pensata apposta per cercare le lexical units dei frame, in WordNet. Non dà alcuna libertà di scelta sul metodo con cui cercare su WordNet, poiché la struttura tipica di una lexical unit é simile a: `<termine>.<pos_tag>`. Di conseguenza, solo ciò che precede il '.' é rilevante. Quindi lo si recupera e lo si ricerca su WordNet, restituendone i risultati.

In [13]:
def get_synset_for_lu(lu_name: str):
    # given a frame name, returns all the relevant synsets
    root = find_root(lu_name.split('.')[0])
    return wn.synsets(root)

La seguente funzione si occupa di creare il contesto per un frame element oppure per una lexical unit. Può procedere in questo modo, poiché nell'interfaccia FrameNet, sia i Frame Elements, che le Lexical Units sono specificate come dizionari (e quindi possiedono la stessa struttura).
In modo simile alle precedenti funzioni per la generazione del contesto, prima di tutto si crea una stringa enorme con definizioni ed esempi per il `felu` (frame-element|lexical unit) in input, dopodiché si usa per creare un modello Bag-of-Words (funzione `utils.word_frequencies`), dopo aver fatto del preprocessing (funzione `utils.preprocess_phrase_nostem`)

In [14]:
def get_context_for_felus(dictionary, felu):
    # given a dictionary, representing frame elements or lexical units
    # returns a disambiguation context (a Bag-of-Words model) made from definitions
    # of the FE/LU and exemplars (if any)
    result = ""
    result += dictionary[felu].definition
    try:
        for ex in dictionary[felu].exemplars:
            #print(ex.text)
            result += ex.text
    except KeyError as ke:
        pass
        #print("No exemplars here")
    return utils.word_frequencies(utils.preprocess_phrase_nostem(result))

#### 3. Valutazione dell'output del sistema

La valutazione dei risultati del mapping è fondamentale. A questo fine è necessario annotare con WN synset ID (ed eventualmente uno o due termini del synset) tutti gli elementi da mappare, e quindi 

- **Frame name** (nel caso si tratti di una multiword expression, come per esempio 'Religious_belief', cercare l'intera multiword su WordNet; se presente annotare con il relativo synset ID; diversamente disambiguare il termine principale, che in generale è il **sostantivo** se l'espressione è composta da NOUN+ADJ, e il **verbo** se l'espressione è composta da VERB+NOUN;
- **Frame Elements (FEs)** del frame; e 
- **Lexical Units (LUs)**.

La correttezza dell'output del sistema sviluppato è da calcolare in rapporto all'annotazione effettuata manualmente. Quindi l'annotazione costituisce un elemento molto importante nello svolgimento dell'esercitazione.

Il programma implementato dovrà quindi fornire anche la funzionalità di valutazione, che confronterà i synset restituiti in output dal sistema con quelli annotati a mano dalla studentessa o dallo studente; su questa base deve essere calcolata l'accuratezza del sistema, semplicemente come rapporto degli elementi corretti sul totale degli elementi.

La seguente lista contiene tutte le annotazioni manuali, di ogni componente del frame, per tutti i frame nel frameset. La variante contenente le 5 parole più prominenti del contesto del synset selezionato si trova in `./results/manual_annotations.txt`. Ogni frame disambiguato possiede la seguente struttura:

```python
{
    'frame_name': ('<Frame_Name>', 'synsetid'),
    'frame_elements: {
        '<element1>': 'synsetid'
    }
    'lexunits': {
        '<lexunit1>': 'synsetid'
    }
}
```

In [15]:
annotated_frames = [{'frame_name': ('Standing_by', 'v.2611373'),'frame_elements': {'Protagonist': 'n.10172793','Salient_entity': 'n.1740','Time': 'n.15122231','Place': 'n.8513718','Duration': 'n.15133621','Manner': 'n.4928903','Expected_request': 'n.7185325','End_point': 'n.15180528','Explanation': 'n.6738281','Depictive': 'v.987071','Purpose': 'v.708980','Co-participant': 'n.10401829','Activity': 'n.407535'},'lexunits': {'stand by.v': 'v.2707125', 'on call.a': 'a.1651196', 'on station.a': 'a.1651196'}}, {'frame_name': ('Being_in_operation', 'n.14008806'), 'frame_elements': {'Device': 'n.3183080','Time': 'n.15122231','Place': 'n.8513718','Duration': 'n.15133621'}, 'lexunits': {'on.prep': 'r.69472','off.prep': 'r.193194','operate.v': 'v.1510827','operational.a': 's.833018'}}, {'frame_name': ('Separating', 'v.2467662'),'frame_elements': {'Whole': 'n.5869584','Parts': 'n.5867413','Part_1': 'n.5671974','Part_2': 'n.5671974','Agent': 'n.6332364','Criterion': 'n.5924920','Depictive': 'v.987071','Manner': 'n.4928903','Degree': 'n.5093890','Means': 'n.3733547','Result': 'n.6333285','Cause': 'n.7326557','Recipients': 'n.6333095','Time': 'n.15122231','Place': 'n.8513718','Instrument': 'n.6332731','Purpose': 'v.708980'},'lexunits': {'bisect.v': 'v.1550817','divide.v': 'v.1458973','part.v': 'v.2030158','partition.v': 'v.1563724','section.v': 'v.1563005','segment.v': 'v.1563005','segregate.v': 'v.494613','split.v': 'v.2467662', 'partition.n': 'n.397953', 'separate.v': 'v.2467662', 'sever.v': 'v.1560731'}}, {'frame_name': ('Volubility', 'n.4651195'), 'frame_elements': {'Speaker': 'n.10630188','Company': 'n.8264897','Text': 'n.6387980','Topic': 'n.6599788','Medium': 'n.6254669','Degree': 'n.5093890','Manner': 'n.4928903','Judge': 'n.10066732'}, 'lexunits': {'effusive.a': 's.806064','glib.a': 's.1874716','laconic.a': 's.547641','loquacious.a': 's.2384077','reticent.a': 's.2383709','silent.a': 's.501820','talkative.a': 's.496938','chatty.a': 's.496422','big mouth.n': None,'brusque.a': 's.640660','curt.a': 's.640660','expansive.a': 's.496938','garrulous.a': 's.2384077','gushing.a': 's.806064','gushy.a': 's.720524','mum.a': 's.501820','voluble.a': 'a.2383831','terse.a': 's.547641','uncommunicative.a': 'a.500569','loquacity.n': 'n.4651382','loudmouth.n': 'n.10274318','chatterbox.n': 'n.9911570','reticence.n': 'n.4652438','reserved.a': 'a.1987341','taciturn.a': 'a.2383380','mute.a': 's.152285','quiet.a': 'a.1918984'}}, {'frame_name': ('Distant_operated_IED', 'n.3565565'), 'frame_elements': {'Bomb': 'n.2866578','Use': 'n.5149325','Type': 'n.5840188','Material': 'n.14580897','Detonator': 'n.3182232','Part': 'n.3892891','Name': 'n.6333653','Time_of_creation': 'n.15122231','Creator': 'n.9614315','Descriptor': 'n.5823747','Target': 'n.10470460'}, 'lexunits': {'RCIED.n': None, 'CWIED.n': None, 'command IED.n': None}}]

Si definisce lo score come:
$$|ctx(w) \cap ctx(s)|+1$$
Per ragioni pratiche (dal momento che la stessa funzione serve per frame name, frame elements e lexical units, vengono passati direttamente i contesti invece che il synset e il frame term).

In [16]:
def score(ctx_s, ctx_w):
    """
        Given contexts from a sense and a word, return the score
    """
    return len(set(ctx_s).intersection(set(ctx_w))) + 1

In [17]:
frames = [fn.frame(i) for i in frameset]

Per evitare di rendere il codice ancora più illegibile, creo un alias `ss_ctx` per calcolare il contesto partendo da un synset, puntandolo alla funzione `get_big_disambiguation_ctx_for_sysnet`

In [18]:
ss_ctx = get_big_disambiguation_ctx_for_sysnet

La seguente funzione si occupa di disambiguare _all'atto pratico_ il frame per intero. Per ogni elemento del frame (nome, elements e lexical units), procede allo stesso modo:  
1. Recupera tutti i possibili synset (`get_synset_for_fn`) per una certa stringa  
2. Calcola gli score per ogni coppia (synset, frame) (calcolandone prima i relativi contesti) e scrivendo il risultato in una lista di tuple `(<synset_scelto>, <score>)`  
3. Ordina la lista per score decrescenti, e prende il primo elemento (il massimo)  
4. Inserisce il risultato in un dizionario strutturato in modo simile ad `annotated_frames` (gli esempi annotati)

In [19]:
def disambiguate_frame(frame):
    result = {'frame_name': "", 'frame_elements': {}, 'lexunits': {}}

    # Disambiguate frame name
    synsets = get_synset_for_fn(frame.name)
    name_ss_scores = [(synset, score(ss_ctx(synset), get_combined_ctx(frame))) for synset in synsets]
    selected_name_ss = sorted(name_ss_scores, key = lambda x: x[1], reverse=True)[0][0]
    result['frame_name'] = (frame.name, f"{selected_name_ss.pos()}.{selected_name_ss.offset()}")

    # Disambiguate frame elements
    for fe in frame.FE.keys():
        fe_ss = get_synset_for_fn(fe)
        fe_ss_scores = [(synset, score(ss_ctx(synset), get_context_for_felus(frame.FE, fe))) for synset in fe_ss]
        selected_fe_ss = sorted(fe_ss_scores, key = lambda x: x[1], reverse=True)[0][0]
        result['frame_elements'][fe] = f"{selected_fe_ss.pos()}.{selected_fe_ss.offset()}"

    # Disambiguate lexical units
    for lu in frame.lexUnit.keys():
        lu_ss = get_synset_for_lu(lu)
        if (lu_ss):
            lu_ss_scores = [(synset, score(ss_ctx(synset), get_context_for_felus(frame.lexUnit, lu))) for synset in lu_ss]
            selected_lu_ss = sorted(lu_ss_scores, key = lambda x: x[1], reverse=True)[0][0]
            result['lexunits'][lu] = f"{selected_lu_ss.pos()}.{selected_lu_ss.offset()}"
        else:
            result['lexunits'][lu] = None
    return result
    

La seguente misura di valutazione si basa sull'uguaglianza dei synset id (tra il frame disambiguato e l'annotazione manuale). Viene calcolata come il conteggio di annotazioni uguali (stesso synset id) sulla totalità degli elementi da annotare (`common_den`)

In [20]:
def equality_evaluation(frame, ground_truth):
    # Given two dictionaries calculate accuracy as overlap between synset ids
    common_den = 1 + len(frame['frame_elements']) + len(frame['lexunits'])
    frame_score = 0
    if frame['frame_name'][1] == ground_truth['frame_name'][1]:
        frame_score += 1/common_den
    for i in frame['frame_elements'].keys():
        if frame['frame_elements'][i] == ground_truth['frame_elements'][i]:
            frame_score += 1/common_den
    
    for j in frame['lexunits'].keys():
        if frame['lexunits'][j] == ground_truth['lexunits'][j]:
            frame_score += 1/common_den
    return frame_score * 100

Calcola l'accuratezza delle annotazioni (troncando il risultato alle ultime 2 cifre decimali per migliorare la leggibilità)

In [21]:
scores = []
for idx in range(len(frames)):
    disamb = disambiguate_frame(frames[idx])
    r = math.trunc(equality_evaluation(disamb, annotated_frames[idx])*100)/100
    print(f"{disamb['frame_name'][0]}: {r}%")
    scores.append(r)

Standing_by: 41.17%
Being_in_operation: 22.22%
Separating: 17.24%
Volubility: 63.88%
Distant_operated_IED: 53.33%


In [22]:
np.mean(scores)

39.568

Tutto sommato il sistema é piuttosto scarso, abbastanza sotto del dare una risposta casuale. In parte potrebbe essere spiegato dalla presenza di  definizioni che esprimono gli stessi concetti, ma sfruttando vocabolari diversi. Differenze che la banale sovrapposizione lessicale non é in grado di cogliere.